In [ ]:
"""Суть задачи 🧬
У нас есть ДНК-последовательность, состоящая всего из 4 букв: A, C, G, T. 
В Python строка хранит каждый символ как Unicode, что занимает 8 байт = 64 бита на символ! 
Это дикая неэффективность, потому что для кодирования 4 вариантов достаточно всего 2 бит.

Математика эффективности:
4 варианта = 2² → нужно 2 бита
Обычная строка: 8 байт = 64 бита на символ
Сжатый вариант: 2 бита на символ
Экономия: (64-2)/64 × 100% = 96.875% (в книге считают 8 бит → 75%)
"""

In [ ]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
# Что происходит: При создании объекта мы сразу сжимаем переданную строку с генами.

In [ ]:
# Шаг 2: Сжатие — магия побитовых операций

def _compress(self, gene: str) -> None:
    self.bit_string = 1  # ← ХИТРОСТЬ: начальная метка!
    
    for nucleotide in gene.upper():
        self.bit_string <<= 2  # Сдвиг влево на 2 бита
        
        if nucleotide == "A":
            self.bit_string |= 0b00  # Добавляем 00
        elif nucleotide == "C":
            self.bit_string |= 0b01  # Добавляем 01
        elif nucleotide == "G":
            self.bit_string |= 0b10  # Добавляем 10
        elif nucleotide == "T":
            self.bit_string |= 0b11  # Добавляем 11

In [ ]:
"""
Почему bit_string = 1? 🤔
Это "стражник" (sentinel) - специальная метка, чтобы отличать начало последовательности. 
Без неё, если ген начинается с A (00), мы бы не знали, где реальные данные.

Пример без метки:

Кодируем "A" → 0b00 → это просто число 0

Как отличить "пустую последовательность" от "последовательности с A"? Никак!

С меткой:

Пусто: 0b1 (просто метка)

"A": сдвигаем 0b1 << 2 = 0b100, добавляем 00 → 0b100 (метка + A)

Побитовые операции в деталях 🔬
Возьмем строку "AT":

Шаг	Действие	bit_string (двоично)	bit_string (десятично)
0	Начало	1	1
1	Сдвиг << 2	100	4
2	Добавляем A (00) через OR	100	4
3	Сдвиг << 2	10000	16
4	Добавляем T (11) через OR	10011	19
Результат: "AT" = 19 в памяти!"""

In [ ]:
# Шаг 3: Распаковка - чтение задом наперед

def decompress(self) -> str:
    gene: str = ""
    # Проходим по битам справа налево с шагом 2
    for i in range(0, self.bit_string.bit_length() - 1, 2):
        bits: int = self.bit_string >> i & 0b11
        
        if bits == 0b00: gene += "A"
        elif bits == 0b01: gene += "C"
        elif bits == 0b10: gene += "G"
        elif bits == 0b11: gene += "T"
    
    return gene[::-1]  # Разворачиваем результат

In [ ]:
"""
Почему читаем справа налево? 🔄
При сжатии мы добавляли новые биты справа (из-за сдвига влево и OR). 
Значит, при чтении самый последний нуклеотид находится в правых битах.

Пример с "AT": 0b10011

Читаем справа: 11 → T

Сдвигаемся левее: 100 → A

Получаем "TA", разворачиваем → "AT"


Визуализация работы 🎨

Исходная строка:    A     T     G
                    ↓     ↓     ↓
Двоичный код:      00    11    10
                    ↓     ↓     ↓
Битовая строка:    00  + 11  + 10
                    ↓
С меткой 1 в начале: 1 00 11 10
                    ↓
В памяти (int):    0b1001110 = 78
"""